<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# 2.6. Electromagnets, induction &amp; inductance

In our previous chapter, we explored Ampere's Law and its applications in understanding the magnetic fields generated by current distributions. 
We concluded with a discussion on hysteresis, which is crucial for understanding the behavior of magnetic materials. 
Building on this foundation, we now delve into the fascinating topics of electromagnets, induction, and inductance.
In doing so, we also introduce alternating current (AC) which is, among other application, used throughout the power grid to deliver electric power to your home.

## 2.6.1. Electromagnets

Electromagnets are devices that generate magnetic fields when an electric current flows through them. 
Two common configurations for electromagnets are solenoids and toroids.

### Solenoids 
A solenoid is a tightly wound helical coil of wire.
Each loop of the coil on its own would produce a circular field but since the loops are so close to each other they start to for a magnetic field similar to that of a bar magnet, i.e. almost uniform inside the coil and a north as well as a south pole. 
The field outside the solenoid is rather weak.
Using Ampère's law we can compute the magnetic field of the solenoid.
Assume the take a rectangular path through the solenoid: first parallel but outside to the coil, then perpendicular going inside the solenoid, hence crossing the loop once, parallel inside the coil, and then going outside again perpendicularly.
Only the portions of the path parallel to the field contribute and the path outside can be neglected as the field is very weak in comparison to inside.
Therefore, we would write
$$ \oint \vec{\bf{B}} \cdot d\vec{\bf{l}} = \int_0^l \vec{\bf{B}} d \vec{\bf{l}} = Bl = \mu_0 I_{enc}$$

for the length $l$ along which we compute the field.
As the enclosed current $I_{enc}$ is given determined by the current $I$ itself and the number of loops $N$, we get
$$ B l = \mu_0 N I \leftrightarrow B = \frac{\mu_0 N I}{l}$$
If we introduce $n$ as the loop density, i.e. the number of loops per unit length, we obtain our final equation:
$$ B = \mu_0 n I$$
Technically, this solution is only true for infinitely long solenoids, but a reasonable approximation for the field inside a long solenoid. 
Solenoids are widely used in applications such as electromagnets, inductors, and magnetic resonance imaging (MRI) machines.

### Toroids  
A toroid is a doughnut-shaped coil of wire (imagine twisting a solenoid into a circle). 
The magnetic field inside a toroid is confined within the core and is given by:  
$$ \oint \vec{\bf{B}} \cdot d\vec{\bf{l}} = B 2 \pi r = \mu_0 I_{enc}$$
$$ B = \frac{\mu_0 N I}{2 \pi r} $$

Toroids are commonly used in power supplies and transformers due to their efficient confinement of magnetic fields.


<!--- todo: images-->



In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from ipywidgets import interact, FloatSlider, IntSlider
from scipy.constants import mu_0, pi

# Global variables for slider parameters
MIN_LENGTH = 1
MAX_LENGTH = 6
STEP_LENGTH = 0.1
INIT_LENGTH = 4

MIN_CURRENT = -2
MAX_CURRENT = 2
STEP_CURRENT = 0.1
INIT_CURRENT = 1

MIN_TURNS = 5
MAX_TURNS = 50
STEP_TURNS = 1
INIT_TURNS = 10

# Fixed diameter of the solenoid
FIXED_DIAMETER = 1.5

# Grid of x, y points
nx, ny = 64, 64
EXTENT = 3
x = np.linspace(-EXTENT, EXTENT, nx)
y = np.linspace(-EXTENT, EXTENT, ny)
X, Y = np.meshgrid(x, y)

# Function to calculate the magnetic field due to a long, straight wire
def calc_b_field(I, X, Y, wire_x=0, wire_y=0):
    r_x = X - wire_x
    r_y = Y - wire_y
    r = np.hypot(r_x, r_y) + 1e-9  # Avoid division by zero

    # Magnetic field magnitude (Ampère's Law)
    B_magnitude = (mu_0 * np.abs(I)) / (2 * pi * r)

    # Direction is tangential to circles around the wire, reversing for negative current
    direction = np.sign(I)
    Bx = direction * -B_magnitude * (r_y / r)
    By = direction * B_magnitude * (r_x / r)

    return Bx, By

# Define global parameters for density scaling
MAX_DENSITY = 5.0

def estimate_max_b_wires():
    Bx_max, By_max = np.zeros((ny, nx)), np.zeros((ny, nx))
    radius = FIXED_DIAMETER / 2
    num_turns = MAX_TURNS
    solenoid_length = MIN_LENGTH
    current = MAX_CURRENT
    length_per_turn = solenoid_length / num_turns if num_turns > 0 else 0
    x_offset = -solenoid_length / 2
    current_bottom = -current
    current_top = current

    for i in range(num_turns):
        x_bottom = x_offset + i * length_per_turn + length_per_turn * 0.25
        x_top = x_offset + i * length_per_turn + length_per_turn * 0.75
        bx_bottom, by_bottom = calc_b_field(current_bottom, X, Y, wire_x=x_bottom, wire_y=-radius)
        Bx_max += bx_bottom
        By_max += by_bottom
        bx_top, by_top = calc_b_field(current_top, X, Y, wire_x=x_top, wire_y=radius)
        Bx_max += bx_top
        By_max += by_top
    return np.hypot(Bx_max, By_max).max()

MAX_B_WIRES = estimate_max_b_wires()
if MAX_B_WIRES == 0:
    MAX_B_WIRES = 1.0 # Avoid division by zero

def plot_solenoid_field_wires(solenoid_length, current, num_turns):
    Bx, By = np.zeros((ny, nx)), np.zeros((ny, nx))
    radius = FIXED_DIAMETER / 2
    length_per_turn = solenoid_length / num_turns if num_turns > 0 else 0
    x_offset = -solenoid_length / 2
    current_bottom = -current
    current_top = current

    for i in range(num_turns):
        x_bottom = x_offset + i * length_per_turn + length_per_turn * 0.25
        x_top = x_offset + i * length_per_turn + length_per_turn * 0.75
        bx_bottom, by_bottom = calc_b_field(current_bottom, X, Y, wire_x=x_bottom, wire_y=-radius)
        Bx += bx_bottom
        By += by_bottom
        bx_top, by_top = calc_b_field(current_top, X, Y, wire_x=x_top, wire_y=radius)
        Bx += bx_top
        By += by_top

    # Calculate current field strength for density scaling
    current_B = np.hypot(Bx, By).max()
    density = MAX_DENSITY * (current_B / MAX_B_WIRES)
    density = np.clip(density, 0.5, MAX_DENSITY * 2) # Ensure reasonable range

    # Plot
    fig, ax = plt.subplots(figsize=(10, 5))
    color = 2 * np.log1p(np.hypot(Bx, By))
    ax.streamplot(x, y, Bx, By, color=color, linewidth=1, cmap=plt.cm.inferno,
                    density=density, arrowstyle='->', arrowsize=1)

    # Draw the solenoid as a single sine wave
    radius = FIXED_DIAMETER / 2
    x_coords = np.linspace(-solenoid_length / 2, solenoid_length / 2, 500)
    y_coords = radius * np.sin(2 * np.pi * num_turns * (x_coords - x_coords.min()) / solenoid_length)
    ax.plot(x_coords, y_coords, 'k-', linewidth=1, label='Solenoid')

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_xlim(-EXTENT, EXTENT)
    ax.set_ylim(-EXTENT / 2 - 0.5, EXTENT / 2 + 0.5)
    ax.set_aspect('equal')
    ax.set_title('Magnetic field of a Solenoid (superposition of single wires)')
    ax.legend()
    plt.show()

interact(plot_solenoid_field_wires,
         solenoid_length=FloatSlider(value=INIT_LENGTH, min=MIN_LENGTH, max=MAX_LENGTH, step=STEP_LENGTH, description='Solenoid Length'),
         current=FloatSlider(value=INIT_CURRENT, min=MIN_CURRENT, max=MAX_CURRENT, step=STEP_CURRENT, description='Current (A)'),
         num_turns=IntSlider(value=INIT_TURNS, min=MIN_TURNS, max=MAX_TURNS, step=STEP_TURNS, description='Number of Turns'))

interactive(children=(FloatSlider(value=4.0, description='Solenoid Length', max=6.0, min=1.0), FloatSlider(val…

<function __main__.plot_solenoid_field_wires(solenoid_length, current, num_turns)>

## 2.6.2. Electromagnetic induction, Lenz's law &amp; Faraday's law

In the previous chapter we saw that (1) an electric current can produce magnetic field and (2) that a magnetic field exerts a force on electric currents as well as moving electric particles.
It were Joseph Henry (1797-1878) and Michael Faraday (1791-1867) who described independently from each other the that a magnetic field can generate an electric current.
The key is that not constant, but a *changing magnetic field induces an electric field*.
We call this phenomena *electromagnetic induction*.

### Magnetic flux

A simple experiment to illustrate electromagnetic induction is moving a bar magnet through a coil. 
The coil is connected to an ammeter and has no electric source connected.
We will see only if the magnet is moved, a current is induced.
The induced current itself is associated with a induced voltage and will generate a magnetic field.

By experimenting with out simple setup, we see that the induced current scales with the magnetic field's magnitude, the loop size, or more specifically it's enclosed area, and the orientation of magnet to loop.
In analogy to the electric flux $\Phi_E$, we can define the *magnetic flux* $\Phi_B$ (capital greek Phi) for a uniform magnetic field:
$$ \Phi_B = B_{\perp} A = B A \cos \theta = \vec{\bf{B}} \vec{\bf{A}} $$

with $\vec{\bf{B}}$ as the magnetic field. 
Further, $\vec{\bf{A}}$ is the vector perpendicular to the area $A$ enclosed by the loop.
Instead of vector multiplication, we can use scala multiplication if we know the angle $\theta$ between $\vec{\bf{B}}$ and $\vec{\bf{A}}$ or use the project of the magnetic field perpendicular to the loop's surface area $B_{\perp}$.
Therefore, and in perfect analogy to the electric flux, the magnetic flux $\Phi_B$ is maximal for an angle $\theta= 90$ degrees and zero for a magnetic field being parallel to the coil.

This equation can be generalized, to arbitrary coil shapes and non-uniform magnetic field by integrating the field over infinitesimal small area elements $d\vec{\bf{A}}$:
$$ \Phi_B = \int \vec{\bf{B}} d \vec{\bf{A}}$$

Note that in contrast to the electric flux $\Phi_E$, a "regular" integral instead of an integral closed surface.
Magnetic flux is expressed in the units *weber* [Wb] = [T m²].


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from ipywidgets import interact
import ipywidgets as widgets

# Grid for the (analogous) uniform magnetic field
EXTENT = 2
x_stream = np.linspace(-EXTENT, EXTENT, 10)
y_stream = np.linspace(-EXTENT, EXTENT, 10)
z_stream = np.linspace(-EXTENT, EXTENT, 10)
X, Y, Z = np.meshgrid(x_stream, y_stream, z_stream)

def plot_magnetic_flux_3D(rect_size, theta_deg, elev, azim):
    theta = np.radians(theta_deg)

    # Define rectangle (initially in yz-plane)
    half_size = rect_size / 2
    corners = np.array([
        [0, -half_size, -half_size],
        [0,  half_size, -half_size],
        [0,  half_size,  half_size],
        [0, -half_size,  half_size]
    ])

    # Rotation matrix (rotate around y-axis)
    R = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])

    rotated_corners = corners @ R.T

    # Compute normal vector (perpendicular to the surface)
    normal_vector = R @ np.array([1, 0, 0])  # Normal originally along x-axis
    center = np.mean(rotated_corners, axis=0)  # Rectangle center
    arrow_length = 1  # Fixed length for visibility

    # Compute magnetic flux: Φ = B • A = B A cos(θ)
    A = rect_size**2  # Area
    B_magnitude = 1  # Uniform (analogous) field magnitude
    flux = B_magnitude * A * np.cos(theta)

    # Create figure
    fig = plt.figure(figsize=(8, 6))
    ax_3d = fig.add_subplot(111, projection='3d')

    # Plot (analogous) uniform magnetic field lines
    # Note: Real magnetic field lines are closed loops. This is a simplified analogy.
    for y_val in np.linspace(-2, 2, 5):
        for z_val in np.linspace(-2, 2, 5):
            ax_3d.plot(x_stream, y_val * np.ones_like(x_stream), z_val * np.ones_like(x_stream),
                        color='blue', linewidth=0.8, alpha=0.6)

    # Plot the rotated rectangle
    ax_3d.plot(*zip(*rotated_corners, rotated_corners[0]), 'r-', linewidth=2)  # Outline

    # Fill rectangle with semi-transparent red
    poly = Poly3DCollection([rotated_corners], color='red', alpha=0.3)
    ax_3d.add_collection3d(poly)

    # Plot normal vector A as a thick red arrow
    ax_3d.quiver(center[0], center[1], center[2],
                    normal_vector[0], normal_vector[1], normal_vector[2],
                    color='red', linewidth=3, arrow_length_ratio=0.2, length=arrow_length)


    # Labels and limits
    ax_3d.set_xlabel('$x$ (direction relative to the field)')
    ax_3d.set_ylabel('$y$')
    ax_3d.set_zlabel('$z$')
    ax_3d.set_xlim(-2, 2)
    ax_3d.set_ylim(-2, 2)
    ax_3d.set_zlim(-2, 2)
    ax_3d.set_title(f'Magnetic Flux: {flux:.2f}\nNote: Magnetic field lines are always closed.')

    # Adjustable camera angles
    ax_3d.view_init(elev=elev, azim=azim)

    plt.show()

# Interactive sliders
interact(plot_magnetic_flux_3D,
         rect_size=widgets.FloatSlider(value=1, min=0.5, max=EXTENT, step=0.1, description="Size"),
         theta_deg=widgets.FloatSlider(value=0, min=0, max=90, step=1, description="Angle (°)"),
         elev=widgets.FloatSlider(value=10, min=0, max=90, step=1, description="Elev"),
         azim=widgets.FloatSlider(value=305, min=0, max=360, step=1, description="Azim"));

interactive(children=(FloatSlider(value=1.0, description='Size', max=2.0, min=0.5), FloatSlider(value=0.0, des…


### Induced voltage, Faraday's law of induction &amp; Lenz's law

We can define the *induced voltage* $V_{ind}$ as the change in magnetic flux over time.
This is the so-called *Faraday's law of induction*:
$$ V_{ind} = - \frac{d\Phi_B}{dt} $$

If the coil used contains $N$ loops, we adapt the equation to:
$$ V_{ind} = - N \frac{d\Phi_B}{dt} $$

Note that we use $V_{ind}$ but in other resources you might find *emf* $\xi$ (greek lower case xi) for the electromotive force. 
Since it is not a force but a voltage, we stick with $V_{ind}$ to minimize potential confusion.

The minus in Faraday's law give already a hint, what *Lenz's law* is about: 

**A change in the magnetic flux through a conducting loop induces a voltage, so that the resulting current generates a magnetic field that opposes the change in magnetic flux**.

Consequently, the induced magnetic field always opposes the magnetic field which generated it and currents oppose the change in flux that induced them.
Furthermore, an associated force, i.e. Lorentz force, will be exerted in the direction opposite to the direction of motion.


### Connecting change in magnetic flux and electric fields: The general form of Faraday's law
We know that  a changing magnetic flux induces a current in a wire.
Further, we know that a flowing current is associated with an electric field.
Hence, a *changing magnetic flux causes an electric field*.

We can describe the relation between change in magnetic flux and electric field produced using the general form of Faraday's law:
$$ \oint \vec{\bf{E}} d \vec{\bf{l}} = - \frac{d\Phi_B}{dt} $$

The right side is Faraday's law of induction (see above) and represents the induced voltage.
The left side is the integral of the electric field along a closed path (closed because we used $\oint$).
We know that the electric field along a path produces a voltage.  
Hence and only logically, the left side represents a voltage as well.
This elegant relation of magnetic flux and electric field is not only valid in conductor but in general (hence the name).

While there resemblance to the laws of electrostatics, the is a stark difference: The *force due to change magnetic fields are non-conservative* and, therefore, there is a path dependence.
In static conditions, i.e. a charge a rest, the electric field begins and ends at charges and, therefore, a integral around a closed path would return zero (in other words moving a charge point $A$ via any path back to point $A$ does not change it potential energy or require any net work).
Here, in the non static condition where we have a changing magnetic field, the integral is not zero and.
Hence, we cannot define potential energies under there conditions and can conclude: *static electric field are conservative fields, electric field due to changing magnetic field are no conservative*.




## 2.6.3. Alternating current, generators &amp; transformers

While batteries generate direct current (DC), electric generators at power plants deliver alternating current (AC).
Further, we saw that in magnetic induction, the current is induced in such way that the induced magnetic field opposes its source.
Therefore, we should think about time-varying currents and voltages.
In AC circuits, the time-varying voltage can be described as a sin-wave with a frequency $f$ and the *peak voltage* $V_0$:
$$ V(t) = V_0 \sin(2\pi f t) = V_0 \sin (\omega t) $$
with $\omega = 2 \pi f$.
Therefore, the voltage alternates between $+V_0$ and $-V_0$.

Consequently, the current $I(t)$ is:
$$ I(t) = \frac{V(t)}{R} = \frac{V_0}{R} \sin(2\pi f t) = I_0 \sin (\omega t) $$

with $I_0$ being the *peak current*.

While the average current over a full period (or multiple of it) is zero, the associated power is non-zero.
Due to the resistance $R$, there will be loses (e.g. heat) and the power transformed is:
$$ P = I^2 R = I_0^2 R \sin^2 (\omega t) $$

Since the average $\sin^2 (\omega t) = \frac{1}{2} $, the average power $\bar{P}$ is:
$$ \bar{P} = \frac{1}{2} I_0^2 R = \frac{1}{2} \frac{V_0^2}{R} $$

One should know that the voltage of an outlet does not refer to the peak voltage $V_0$ but the *root-mean-squared (RMS)* voltage.
To obtain $I_{RMS}$ and $V_{RMS}$, we use the relation between peak and squared average $\bar{I^2} = \frac{1}{2} I_0^2$ &amp; $\bar{V^2} = \frac{1}{2} V_0^2$:

 $$ I_{rms} = \sqrt{\bar{I^2}} = \sqrt{\frac{1}{2} I_0^2} = \frac{I_0}{\sqrt{2}} $$
$$ V_{rms} = \sqrt{\bar{V^2}} = \sqrt{\frac{1}{2} V_0^2} = \frac{V_0}{\sqrt{2}} $$

Therefore, for a 220V outlet, the peak voltage is $\sqrt{2} \cdot 220 \textrm{V} \approx 311 \textrm{V}  $.

The average power w.r.t. RMS voltage and current is:
$$ \bar{P} =  I_{RMS} V_{RMS}= I_{RMS}^2 R = \frac{V_{RMS}^2}{R} $$

### Induction in a moving conductor
Lets imagine an uniform magnetic field $\vec{\bf{B}}$ and an U-shaped wire perpendicular to it.
If we put a conducting rod with length $l$ across the U-shaped wire and move it with the constant speed $v$ along the U, in the induced voltage is:
$$ V_{ind} = \frac{d\Phi_B}{dt} = \frac{B dA}{dt} $$

The area $dA$ is not dependent on the U-shaped conductor but on the length of the rod and the distance $dx$ it travels within $dt$ which we get from $v = \frac{dx}{dt}$:
$$ dA = l dx = l v dt$$

Therefore, we obtain the simple equation:
$$ V_{ind} = \frac{B l v dt}{dt} = B l v $$

This equation is valid for all configuration in which $v$, $B$, and $l$ are perpendicular to each other.

Not that this equation could also have been obtained by considering the forces at play:
Without considering the connected U-shaped wire, the charges inside the rod would effectively also move with $v$ perpendicular to $B$.
We know that the force is then simply $F = q v B$.
As the absent U-shaped wire prevent a current running in a closed loop, the charges get simply separated and the induced voltage is measure from one end of the rod to the other.
The work required to move the electrons from one to the other side of the rod is $W = F l = q v B l$. 
We also defined voltage as work per unit charge.
Hence the induced voltage is $V_{ind} = \frac{W}{q} = \frac{q v B l}{q} = v B l $ which is the same as the equation above.


### Electric generators
Electric generators produce alternating currents (AC) from mechanical energy by leveraging electromagnetic induction.

The most simple generators have a static magnetic field $\vec{\bf{B}}$ generated by magnet and, inside the field, a rotating *armature* with loops of wire wounded around it.
As the connected axle rotate the armature, a voltage $V_{ind}$ is induced in the rotating coil (often refereed to as *emf*, although not a force but a voltage).
To let a current flow, the end of the coil are connected to slip rings on the axle, enabling the generator to produce a current, ergo electric energy.

In a uniform magnetic field, the induced voltage $V_{ind}$ is:
$$ V_{ind} = - \frac{d\Phi_B}{dt} = - \frac{d}{dt} \left( \int \vec{\bf{B}} d\vec{\bf{A}} \right) = - \frac{d}{dt}(B A \cos \theta) $$
with $\theta$ the angle between the magnetic field and the vector $\vec{\bf{A}}$ perpendicular to the area $A$ of the loop.

The equation does require us to differentiate w.r.t. time. 
Only the angle $\theta$ is time dependent (coil rotates, hence, orientation w.r.t. the magnetic field changes, field and area are constant).
Assuming the initial angle is $\theta_0 = 0$ and a constant rotation velocity $\omega = \frac{d\theta}{dt}$ we can write:
$$ V_{ind} = - B A \frac{d \cos (\omega t)}{dt} = B A \omega \sin (\omega t) $$

We can extent this solution to $N$ loop of a coil and introduce the maximally induced voltage $V_{ind, 0} = N B A \omega$ :
$$ V_{ind} = N B A \omega \sin (\omega t) = V_{ind, 0} \sin (\omega t) $$

This is a simple AC voltage and entities such as RMS can be computed as stated above.

The concept of a generator, i.e. converting mechanical into electrical energy, can be inverted to obtain an electric *motor*.

There is another important effect to be considered when talking about (moving) conductors in (changing) magnetic field:
When conductors move inside a magnetic field or the magnetic field itself changes, loops of electrical current are induced, the so-called *eddy currents*.
They arises from Faraday's law of induction. 
Essentially, when a magnetic field around any conductor (wire, wheel of steel, etc.) changes, the induced voltage drives the flow of electrical current within the conductor.
These currents circulate in closed loops, often resembling swirling patterns, hence the name "eddy currents." 
Further and in accordance to Lenz's law, they circulate in the direction creating a magnetic field opposing its source.
Eddy currents have various effects, including the generation of heat due to the conductor's resistance, which is utilized in induction heating. 
They also produce a braking effect, as the magnetic field they create opposes the change that caused them, a principle used in electromagnetic brakes. 
Additionally, eddy currents find applications in non-destructive testing (detect flaws in metal materials) or metal detector (detect hidden pieces of metal).
However, in devices like generators and transformers, eddy currents can lead to energy losses, reducing efficiency. 
When a solid conductive core within a generator is subjected to a changing magnetic field, eddy currents are induced, circulating within the core material. 
These currents, flowing through the core's resistance, generate heat, leading to energy dissipation and a reduction in overall efficiency. 
*Lamination* mitigates this by constructing the core from thin sheets of electrically insulated conductive material, commonly steel. 
This layered structure disrupts the path of eddy currents, substantially decreasing their magnitude. 
By dividing the core into numerous thin, insulated layers, the available pathways for eddy current flow are restricted, minimizing the amount of circulating current.


### Transformers
With transformers, we can increase of decrease the voltage in an AC circuit.
In essence, a transformer consists of a *primary* and *secondary* coil, usually sharing an iron core to increase the magnetic field.
Note that the two coils do not form a closed circuit, but only their proximity is essential for the transformer to work.
In ideal conditions, i.e. no energy loses, an AC voltage in the primary coil, will induce the voltage $V_2$ in the second coil with $N_2$ loops du to a change in the magnetic flux $\Phi_B$:
$$ V_2 = N_2 \frac{d \Phi_B}{dt} $$

There is also a *self-induction* in the primary coil because the field changes with in it as well:
$$ V_1 = N_1 \frac{d \Phi_B}{dt} $$

Dividing both equation yields the simple relation of voltages and number of loops per coil:
$$ \frac{V_2}{V_1} = \frac{N_2}{N_1} $$

There are two types of transformers, depending on the ratio of the loop numbers:
* step-up transformer: For $N_2 > N_1$ the voltage $V_2$ will be larger than $V_1$
* step-down transformer: For $N_2 < N_1$ the voltage $V_2$ will be smaller than $V_1$

Naturally, the conservation of energy applies to transformers.
Assuming no loses, the power in the primary and secondary loop is equal $P_1 = P_2$ and, therefore, the current will be decreases (increased) if the voltages is decreased (increased) by the transformer:
$$ I_1 V_1 = I_2 V_2 \leftrightarrow \frac{I_1}{I_2} = \frac{N_2}{N_1}$$

Transformers are found everywhere in daily life, e.g. in the power grid and (wireless) chargers.

Note that, direct currents (DC) would not work with transformers as the field ist not (constantly) changing.

### Mutual inductance
Let's build on our knowledge of generators and transformers to investigate *mutual inductance*.
Again, we place two coils close to each other.
If the current is changing in coil $1$, then there will be a voltage induced in coil $2$ according to $ V_2 = N_2 \frac{d \Phi_B}{dt} $.
However, it might be handy to relate the induced voltage $V_2$, directly to the change in current in coil $1$.
To that end, we define the proportionality constant *mutual inductance* $M_{21}$ as:
$$ M_{21} = \frac{N_2 \Phi_{21}}{I_1} $$

Therefore, the voltage induced in coil $2$ can be expressed as:
$$ V_2 = N_2 \frac{d \Phi_{21}}{dt}  = - M_{21} \frac{d I_1}{dt} $$

Consequently, a changing current in coil $2$ would induce the voltage $V_1$ in coil $1$ with the proportionality constant $M_{12}$
$$ V_1  = - M_{12} \frac{d I_2}{dt} $$

The mutual inductance only depends on geometric variables, i.e. the spacing between the coils and their individual shape, size, and number of loops.
Therefore, we can use: 
$$ M = M_{21} = M_{12} $$

and simplify the equations to:
$$ V_1  = - M \frac{d I_2}{dt} \quad \textrm{\&} \quad V_2  = - M \frac{d I_1}{dt} $$

Inductance (regardless if "mutual" or "self") is expressed in units of *henry* [H] = [V s / A] = [$\Omega$ s]

### Self-inductance &amp; inductors

Even for a single coil, we found the phenomenon of inductance.
This *self-inductance* stems from the fact that as the changing current passes through the coil, a voltage is induced in the coil itself (opposed direction).
In line to mutual inductance, we define self-inductance $L$ as a proportionality constant of relate the induced voltage $V_{ind}$ directly to the change in current:
$$ L = \frac{N \Phi_b}{I} $$
$$ V_{ind} = - N \frac{d \Phi_B}{dt} = -L \frac{d I}{dt} $$

Self-inductance is often referred to as simply inductance and, as stated before, measured in henry [H].
The inductance of a coil depends on its geometry.
Inductors are devices used in electric circuits and represent along resistors and capacitors the core building blocks.

<!--- todo: symbol inductor -->


### Energy stored in the magnetic field

Let's assume we run a changing current through an inductor with inductance $L$.
The power would be:
$$ P = I V_{ind} = I L \frac{d I}{dt} $$

Note that the minus sign is omitted as we supply power to oppose the induced voltage.
As power also defined as work per unit time, we can write that the work $dW$ performed during $dt$ is:
$$dW = P dt = I L \frac{dI}{dt} dt = I L dI $$
The work required in total is therefore:
$$ W = \int_0^I d W = \int_0^I I L dI = \frac{1}{2} I^2 L $$ 

As the energy stored has the same magnitude as the work we obtain:
$$ U = \frac{1}{2} L I^2$$

This energy is stored in the magnetic field.
If we think about energy stored in an electric field of a capacitor, we find certain similarities:
$$ U = \frac{1}{2} C V^2$$

<!--- todo: images -->

<!--- todo: applications -->